## LionForests Multi Label Experiments: Water Quality Dataset

In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

In [2]:
from sklearn.metrics import f1_score, precision_score
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import warnings
import random
import numpy as np
from datasets.dataset import Dataset
import pandas as pd
from utilities.dummy_utilizer import DummyUtilizer
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from algorithms.MARLENA.marlena.marlena.marlena import MARLENA
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.CHIRPS.structures import data_container
import algorithms.CHIRPS.routines as rt
import algorithms.CHIRPS.structures as strcts
from scipy import sparse
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer

np.seterr(invalid='ignore')
warnings.filterwarnings("ignore")

Load Dataset

In [3]:
water_quality = Dataset()
X, y, feature_names, label_names = water_quality.load_water_quality()

Initialize LionForests

In [4]:
from lionforestsmulti import LionForests
lf = LionForests(None, False, None, feature_names, label_names)#,'x2'])

In [5]:
parameters = [{
    'max_depth': [10],
    'max_features': [None],
    'bootstrap': [True],
    'min_samples_leaf' : [1],
    'n_estimators': [100]
}]
lf.fit(X, list(y), parameters, True)

Best RF model for this dataset

In [6]:
lf.model

RandomForestClassifier(max_depth=10, max_features=None, n_jobs=-1,
                       random_state=0)

Example explanation for option "all predicted labelsets"

In [7]:
lf.explain_n_wise(X[2],'all',True, method='1')[0]

'if 3.224<=std_temp<=3.321 & 23.121<=std_pH<=23.572 & 2.699<=conduct<=2.783 & 3.625<=o2<=3.666 & 4.124<=o2sat<=4.186 & 0.622<=co2<=0.758 & 2.936<=hardness<=2.955 & 0.432<=no2<=0.489 & 2.976<=no3<=3.574 & 0.095<=nh4<=0.099 & 0.263<=po4<=0.326 & 0.771<=cl<=0.782 & 3.171<=sio2<=3.188 & 0.716<=kmno4<=0.885 & 0.703<=k2cr2o7<=0.717 & 0.243<=bod<=0.252 then 25400 29600 17300 34500 49700 50390 55800 59300 37880'

Example explanation for option "per label"

In [8]:
lf.explain_n_wise(X[2],'per label',True, method='1')['17300'][0]

'if 3.224<=std_temp<=3.321 & 23.121<=std_pH<=23.572 & 2.699<=conduct<=2.783 & 3.625<=o2<=3.666 & 4.124<=o2sat<=4.186 & 2.936<=hardness<=3.014 & 0.432<=no2<=0.489 & 2.976<=no3<=3.574 & 0.095<=nh4<=0.099 & 0.263<=po4<=0.326 & 0.771<=cl<=0.782 & 3.171<=sio2<=3.188 & 0.676<=kmno4<=0.885 & 0.703<=k2cr2o7<=0.72 & 0.243<=bod<=0.252 then 17300'

Example explanation for option "frequent pairs"

In [9]:
lf.explain_n_wise(X[2],'frequent pairs',7, True, method='1')[('29600','17300')][0]

'if 3.224<=std_temp<=3.321 & 23.121<=std_pH<=23.572 & 2.699<=conduct<=2.783 & 3.625<=o2<=3.666 & 4.124<=o2sat<=4.186 & 0.622<=co2<=0.758 & 2.936<=hardness<=2.955 & 0.432<=no2<=0.489 & 2.976<=no3<=3.574 & 0.095<=nh4<=0.099 & 0.263<=po4<=0.326 & 0.771<=cl<=0.782 & 3.171<=sio2<=3.188 & 0.716<=kmno4<=0.885 & 0.703<=k2cr2o7<=0.717 & 0.243<=bod<=0.252 then 29600 17300'

## Experiments

1. Comparison between LF variations:

In [9]:
def measure(X_train, X_test, y_train, y_test, feature_names, label_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [None],
        'bootstrap': [True],
        'min_samples_leaf': [1],
        'n_estimators': [100]
    }]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    lf = LionForests(None, False, scaler, feature_names, label_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, label_names, lf, task, random_state=10):

        # LionForests
        def lf_rule_all(instance):
            temp = lf.explain_n_wise(instance, 'all')[5]
            rule = {}
            for key, value in temp.items():
                rule[key] = [['<=', value[1]], ['>', value[0]]]
            return rule

        def lf_rule_per_label(instance):
            temp = lf.explain_n_wise(instance, 'per label')
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        def lf_rule_pairs(instance):
            temp = lf.explain_n_wise(
                instance, 'frequent pairs', len(label_names))
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        return {'lf-a': lf_rule_all, 'lf-l': lf_rule_per_label, 'lf-p': lf_rule_pairs}

    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, label_names, lf,
                                'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-a': 0, 'lf-l': 0, 'lf-p': 0}
    rule_length = {'lf-a': 0, 'lf-l': 0, 'lf-p': 0}
    f_precision = {'lf-a': [], 'lf-l': [], 'lf-p': []}
    time_response = {'lf-a': [], 'lf-l': [], 'lf-p': []}
    rules = {'lf-a': [], 'lf-l': [], 'lf-p': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
    skippedkataskeywastwnplunthriousunistounskipmadame = 0
    for tesd_ind in range(len(test)):
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            #print(tesd_ind,'sunexise madam')
            for name, method in rule_generator.items():
                if name == 'lf-a':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp_lf:
                        res = rule_cov_LF(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(y_test_temp[tesd_ind]), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp_lf)
                    rules[name].append(rule)
                    len_rule = len(rule)
                elif name == 'lf-l':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'lf-p':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        labeles = []
                        for kk in key:
                            labeles.append(list(label_names).index(kk))
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                temp_prediction = [y_test_temp[tesd_ind][j] for j in range(
                                    len(y_test_temp[tesd_ind])) if j in labeles]
                                temperatura = [y_test_temp[co][j] for j in range(
                                    len(y_test_temp[co])) if j in labeles]
                                precision.append(
                                    [temp_prediction, temperatura])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, 0], precision[:, 1], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)

                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds=folds+1

Results

In [21]:
full_coverage = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
rule_length = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
f_precision ={'lf-a':[], 'lf-l':[], 'lf-p':[]}
f_time = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

lf-a | 0.0096  0.000 | 16.5372 0.421 | 1.0000  0.000 | 0.3770  0.003
lf-l | 0.0096  0.000 | 60.7630 6.220 | 1.0000  0.000 | 1.3374  0.122
lf-p | 0.0096  0.000 | 153.3407 16.756 | 1.0000  0.000 | 2.8663  0.290


2. Comparison between LF-a and multi-label explainability techniques

In [10]:
def measure(X_train, X_test, y_train, y_test, feature_names, label_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [None],
        'bootstrap': [True],
        'min_samples_leaf': [1],
        'n_estimators': [100]
    }]
    lf = LionForests(None, False, None, feature_names, label_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, label_names, lf, task, random_state=10):

        # LionForests
        def lf_rule_all(instance):
            temp = lf.explain_n_wise(instance, 'all')[5]
            rule = {}
            for key, value in temp.items():
                rule[key] = [['<=', value[1]], ['>', value[0]]]
            return rule

        gt = GlobalSurrogateTree(
            train, predictions, feature_names, task, random_state)
        print('    GT Ready')
        lt = LocalSurrogateTree(
            train, predictions, feature_names, task, 150, random_state)

        def marlena(instance):
            m1 = MARLENA(neigh_type='mixed', random_state=42)
            i2e = pd.Series(instance, index=feature_names)
            X2E = pd.DataFrame(train, columns=feature_names)
            rule, _, _, _, _ = m1.extract_explanation(i2e, X2E, lf.model, feature_names, [],
                                                      label_names, k=10, size=50, alpha=0.7)

            path = rule.split('->')[0][1:-2]
            prediction = rule.split('->')[1][1:]
            conjuctions = {}
            if '{}' in rule:
                print(rule)
            else:
                for conj in path.split('\n'):
                    temp_conj = conj.strip(',')
                    if temp_conj[0] == ' ':
                        temp_conj = temp_conj[1:]
                    if '>' in temp_conj:
                        temp = temp_conj.split(' > ')
                        if temp[0] in conjuctions:
                            position = 0
                            if len(conjuctions[temp[0]]) == 2:
                                if conjuctions[temp[0]][1][0] == '>':
                                    position = 1
                            if conjuctions[temp[0]][position][0] == '>':
                                if conjuctions[temp[0]][position][1] > float(temp[1]):
                                    conjuctions[temp[0]][position][1] = float(
                                        temp[1])
                            elif conjuctions[temp[0]][0][0] == '<=':
                                conjuctions[temp[0]].append(
                                    ['>', float(temp[1])])

                        else:
                            conjuctions[temp[0]] = [['>', float(temp[1])]]
                    else:
                        temp = temp_conj.split(' <= ')
                        if temp[0] in conjuctions:
                            position = 0
                            if len(conjuctions[temp[0]]) == 2:
                                if conjuctions[temp[0]][1][0] == '<=':
                                    position = 1

                            if conjuctions[temp[0]][position][0] == '<=':
                                if conjuctions[temp[0]][position][1] <= float(temp[1]):
                                    conjuctions[temp[0]][position][1] = float(
                                        temp[1])
                            elif conjuctions[temp[0]][0][0] == '>':
                                conjuctions[temp[0]].append(
                                    ['<=', float(temp[1])])
                        else:
                            conjuctions[temp[0]] = [['<=', float(temp[1])]]
            local_prediction = np.zeros((len(label_names)), dtype=int)
            for label in range(len(label_names)):
                if label_names[label] in prediction:
                    local_prediction[label] = 1
            return (conjuctions, local_prediction)

        return {'lf-a': lf_rule_all, 'gs': gt.rule, 'ls': lt.rule, 'ma': marlena}

    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, label_names, lf,
                                'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-a': 0, 'ls': 0, 'gs': 0, 'ma': 0}
    rule_length = {'lf-a': 0, 'ls': 0, 'gs': 0, 'ma': 0}
    f_precision = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}
    time_response = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}
    rules = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        # or trees in lf.model.estimators_:
        #   print(trees.predict_proba([x_test_temp_lf[tesd_ind]]))
        # rint(y_test_temp[tesd_ind])
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            #print(tesd_ind,'sunexise madam')
            for name, method in rule_generator.items():
                if name == 'lf-a':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp_lf:
                        res = rule_cov_LF(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(y_test_temp[tesd_ind]), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp_lf)
                    rules[name].append(rule)
                    len_rule = len(rule)
                elif name == 'ls' or name == 'gs' or name == 'ma':
                    ts = time.time()
                    rule, prediction = method(x_test_temp[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp:
                        res = rule_cov(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(prediction), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp)
                    rules[name].append(rule)
                    len_rule = len(rule)
                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10, random_state = 77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds = folds + 1

Results

In [13]:
full_coverage = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
rule_length = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
f_precision ={'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
f_time = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

lf-a | 0.0095  0.000 | 16.3655 0.404 | 1.0000  0.000 | 0.9377  0.042
gs | 0.0193  0.004 | 8.7795 0.365 | 0.6645  0.048 | 0.0004  0.000
ls | 0.0294  0.004 | 6.6755 0.207 | 0.6545  0.026 | 1.4057  0.034
ma | 0.0473  0.009 | 5.9790 0.335 | 0.6242  0.050 | 2.0390  0.011


3. Comparison between LF-l and "per label" explainability techniques (CHIRPS/ANCHORS)

In [51]:
def measure(X_train, X_test, y_train, y_test, feature_names, label_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [None],
        'bootstrap': [True],
        'min_samples_leaf': [1],
        'n_estimators': [100]
    }]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    lf = LionForests(None, False, scaler, feature_names, label_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, label_names, lf, task, random_state=10):

        def lf_rule_per_label(instance):
            temp = lf.explain_n_wise(instance, 'per label')
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        inddd = 0
        anchors = {}
        for label_name in label_names:
            explainer = AnchorTabularExplainer(
                ['not '+label_name, label_name], feature_names, train)
            anchors[inddd] = explainer
            inddd += 1

        def anchors_method(instance):
            prediction = lf.model.predict([instance])[0]
            rules = {}
            for pred in range(len(prediction)):
                if prediction[pred] == 1:
                    explainer = anchors[pred]

                    def my_model(x):
                        return lf.model.predict(x)[:, pred]
                    exp = explainer.explain_instance(
                        instance, my_model, threshold=0.95)
                    anchors_dict = {}
                    for i in exp.names():
                        terms = i.split(' ')
                        if len(terms) == 3:
                            anchors_dict[terms[0]] = [
                                [terms[1], float(terms[2])]]
                        else:
                            anchors_dict[terms[2]] = [[terms[3], float(terms[4])], [
                                terms[1], float(terms[0])]]
                    rules[label_names[pred]] = anchors_dict
            return rules

        chts = time.time()
        chirps = {}
        inddd = 0
        for label_name in label_names:
            explainer = AnchorTabularExplainer(
                ['not '+label_name, label_name], feature_names, train)
            # CHIRPS =======================================================================================
            project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
            temp_frame = pd.DataFrame(np.hstack((train, y_train[:, inddd].reshape(
                len(y_train), 1))), columns=feature_names+['class'])
            temp_frame['class'] = temp_frame['class'].astype(int)
            #temp_frame = temp_frame.replace({"class": {1: 2}})
            #temp_frame = temp_frame.replace({"class": {0: 1}})

            mydata = data_container(
                data=temp_frame, class_col='class', var_names=feature_names,
                project_dir=project_dir, save_dir='WQ'+str(inddd), random_state=random_state)
            meta_data = mydata.get_meta()
            f_walker = strcts.classification_trees_walker(
                forest=model, inddd=inddd, meta_data=meta_data)
            f_walker.forest_walk(instances=test, labels=model.predict(test)[
                                 :, inddd], forest_walk_async=False)

            explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                                   forest=model,
                                                   # any representative sample can be used
                                                   sample_instances=sparse.csr_matrix(
                                                       train),
                                                   sample_labels=predictions[:, inddd],
                                                   meta_data=meta_data)
            explanations.run_explanations(target_classes=model.predict(test)[:, inddd],  # we're explaining the prediction, not the true label!
                                          explanation_async=False,
                                          random_state=random_state,
                                          which_trees='majority',
                                          alpha_paths=0.0,
                                          support_paths=0.1,
                                          score_func=1,
                                          precis_threshold=0.8,
                                          disc_path_bins=2,
                                          merging_bootstraps=10,
                                          pruning_bootstraps=10,
                                          delta=0.2)
            chirps[inddd] = explanations
            inddd += 1
        chte = (time.time() - chts)/len(test)

        def chirps_method(instance, idx):
            prediction = lf.model.predict([instance])[0]
            rules = {}
            for pred in range(len(prediction)):
                if prediction[pred] == 1:
                    chirps_dict = {}
                    explanations = chirps[pred]
                    for i in explanations.explainers[idx].pruned_rule:
                        if i[1]:
                            chirps_dict[i[0]] = [['<=', float(i[2])]]
                        else:
                            chirps_dict[i[0]] = [['>', float(i[2])]]
                    rules[label_names[pred]] = chirps_dict
            return rules, 0, chte

        return {'lf-l': lf_rule_per_label, 'an': anchors_method, 'ch': chirps_method}

    interpretation = techniques(lf.model, train, y_train, predictions,
                                test, feature_names, label_names, lf, 'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-l': 0, 'an': 0, 'ch': 0}
    rule_length = {'lf-l': 0, 'an': 0, 'ch': 0}
    f_precision = {'lf-l': [], 'an': [], 'ch': []}
    time_response = {'lf-l': [], 'an': [], 'ch': []}
    rules = {'lf-l': [], 'an': [], 'ch': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)

    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        # or trees in lf.model.estimators_:
        #   print(trees.predict_proba([x_test_temp_lf[tesd_ind]]))
        # rint(y_test_temp[tesd_ind])
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:

            for name, method in rule_generator.items():
                if name == 'lf-l':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'an':
                    ts = time.time()
                    rule = method(x_test_temp[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp:
                            res = rule_cov(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'ch':
                    rule, op, te = method(x_test_temp[tesd_ind], tesd_ind)
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp:
                            res = rule_cov(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)

                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame


In [52]:
y = y.astype('int64')

In [ ]:
total_results = []
kf = KFold(n_splits=10, random_state=77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    test_size.append(len(X_test))
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    total_results.append(results)
    folds=folds+1

Results

In [54]:
full_coverage = {'lf-l':[], 'an':[], 'ch':[]}
rule_length = {'lf-l':[], 'an':[], 'ch':[]}
f_precision = {'lf-l':[], 'an':[], 'ch':[]}
f_time = {'lf-l':[], 'an':[], 'ch':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

lf-l | 0.0091  0.000 | 57.6415 5.714 | 1.0000  0.000 | 1.3705  0.112
an | 0.0372  0.011 | 31.0481 3.474 | 0.9628  0.023 | 302.0667  44.434
ch | 0.3754  0.046 | 10.3594 1.366 | 0.7583  0.046 | 8.2391  0.500
